In [35]:
import pandas as pd
import numpy as np

In [36]:
olist_orders = pd.read_csv("../data/raw/archive/olist_orders_dataset.csv")
olist_order_items = pd.read_csv("../data/raw/archive/olist_order_items_dataset.csv")
olist_customers = pd.read_csv("../data/raw/archive/olist_customers_dataset.csv")

# Processamento

- Junção das Tabelas
- Filtro de Variáveis
- Construção de features:
    - Faturamento Total por Cliente
    - Quantidade Total de Pedidos por Cliente

Junção das Tabelas

In [37]:
customers_orders = olist_customers.merge(olist_orders, how="inner", on="customer_id")
customers_orders_items = customers_orders.merge(olist_order_items, how="inner", on="order_id")

In [38]:
customers_orders_items

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00,1,a9516a079e37a9c9c36b9b78b10169e8,7c67e1448b00f6e969d365cea6b010ab,2017-05-22 15:22:12,124.99,21.88
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06 00:00:00,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-01-18 20:58:32,289.00,46.48
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13 00:00:00,1,bd07b66896d6f1494f5b86251848ced7,7c67e1448b00f6e969d365cea6b010ab,2018-06-05 16:19:10,139.94,17.79
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10 00:00:00,1,a5647c44af977b148e0a3a4751a09e2e,7c67e1448b00f6e969d365cea6b010ab,2018-03-27 16:31:16,149.94,23.36
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15 00:00:00,1,9391a573abe00141c56e38d84d7d5b3b,4a3ca9315b744ce9f8e9374361493884,2018-07-31 10:10:09,230.00,22.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112645,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,sao paulo,SP,6760e20addcf0121e9d58f2f1ff14298,delivered,2018-04-07 15:48:17,2018-04-07 16:08:45,2018-04-11 02:08:36,2018-04-13 20:06:37,2018-04-25 00:00:00,1,ccb4503d9d43d245d3b295d0544f988b,527801b552d0077ffd170872eb49683b,2018-04-12 16:08:45,74.90,13.88
112646,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,taboao da serra,SP,9ec0c8947d973db4f4e8dcf1fbfa8f1b,delivered,2018-04-04 08:20:22,2018-04-04 08:35:12,2018-04-05 18:42:35,2018-04-11 18:54:45,2018-04-20 00:00:00,1,9ede6b0570a75a4b9de4f383329f99ee,3fd1e727ba94cfe122d165e176ce7967,2018-04-10 08:35:12,114.90,14.16
112647,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE,fed4434add09a6f332ea398efd656a5c,delivered,2018-04-08 20:11:50,2018-04-08 20:30:03,2018-04-09 17:52:17,2018-05-09 19:03:15,2018-05-02 00:00:00,1,7a5d2e1e131a860ae7d18f6fffa9d689,d9e7e7778b32987280a6f2cb9a39c57d,2018-04-12 20:30:03,37.00,19.04
112648,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS,e31ec91cea1ecf97797787471f98a8c2,delivered,2017-11-03 21:08:33,2017-11-03 21:31:20,2017-11-06 18:24:41,2017-11-16 19:58:39,2017-12-05 00:00:00,1,f819f0c84a64f02d3a5606ca95edd272,4869f7a5dfa277a7dca6462dcf3b52b2,2017-11-09 21:15:51,689.00,22.07


Filtro de Variáveis

In [39]:
customers_orders_items.columns

Index(['customer_id', 'customer_unique_id', 'customer_zip_code_prefix',
       'customer_city', 'customer_state', 'order_id', 'order_status',
       'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'order_item_id', 'product_id',
       'seller_id', 'shipping_limit_date', 'price', 'freight_value'],
      dtype='object')

Vamos pegar por consumidor o total que ele gastou (o faturamento da empresa) e o total de pedidos que foram feitos.

Colunas usadas:
- customer_unique_id
- order_id
- order_status
- price
- freight_value

Agora vamos pegar essa colunas:

In [40]:
customers_orders_items.drop(columns=[
                                        i for i in customers_orders_items.columns if i not in['customer_unique_id', 'order_id', 'order_status', 'price', 'freight_value']
                                    ],
                            inplace=True)

In [41]:
customers_orders_items

,customer_unique_id,order_id,order_status,price,freight_value
0,861eff4711a542e4b93843c6dd7febb0,00e7ee1b050b8499577073aeb2a297a1,delivered,124.99,21.88
1,290c77bc529b7ac935b93aa66c333dc3,29150127e6685892b6eab3eec79f59c7,delivered,289.00,46.48
2,060e732b5b29e8181a18229c7b0b2b5e,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,139.94,17.79
3,259dac757896d24d7702b9acbbff3f3c,951670f92359f4fe4a63112aa7306eba,delivered,149.94,23.36
4,345ecd01c38d18a9036ed96c73b8d066,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,230.00,22.25
...,...,...,...,...,...
112645,1a29b476fee25c95fbafc67c5ac95cf8,6760e20addcf0121e9d58f2f1ff14298,delivered,74.90,13.88
112646,d52a67c98be1cf6a5c84435bd38d095d,9ec0c8947d973db4f4e8dcf1fbfa8f1b,delivered,114.90,14.16
112647,e9f50caf99f032f0bf3c55141f019d99,fed4434add09a6f332ea398efd656a5c,delivered,37.00,19.04
112648,73c2643a0a458b49f58cea58833b192e,e31ec91cea1ecf97797787471f98a8c2,delivered,689.00,22.07


In [42]:
customers_orders_items.order_status.unique()

array(['delivered', 'invoiced', 'shipped', 'processing', 'canceled',
       'unavailable', 'approved'], dtype=object)

Retirar as linhas que apresentam 'canceled' e 'unavailable' da coluna 'order_status'

In [43]:
customers_orders_items = customers_orders_items[(customers_orders_items.order_status != 'unavailable') & (customers_orders_items.order_status != 'canceled')]

In [44]:
customers_orders_items

,customer_unique_id,order_id,order_status,price,freight_value
0,861eff4711a542e4b93843c6dd7febb0,00e7ee1b050b8499577073aeb2a297a1,delivered,124.99,21.88
1,290c77bc529b7ac935b93aa66c333dc3,29150127e6685892b6eab3eec79f59c7,delivered,289.00,46.48
2,060e732b5b29e8181a18229c7b0b2b5e,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,139.94,17.79
3,259dac757896d24d7702b9acbbff3f3c,951670f92359f4fe4a63112aa7306eba,delivered,149.94,23.36
4,345ecd01c38d18a9036ed96c73b8d066,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,230.00,22.25
...,...,...,...,...,...
112645,1a29b476fee25c95fbafc67c5ac95cf8,6760e20addcf0121e9d58f2f1ff14298,delivered,74.90,13.88
112646,d52a67c98be1cf6a5c84435bd38d095d,9ec0c8947d973db4f4e8dcf1fbfa8f1b,delivered,114.90,14.16
112647,e9f50caf99f032f0bf3c55141f019d99,fed4434add09a6f332ea398efd656a5c,delivered,37.00,19.04
112648,73c2643a0a458b49f58cea58833b192e,e31ec91cea1ecf97797787471f98a8c2,delivered,689.00,22.07


Feature - Faturamento total por cliente

In [45]:
values_by_customer = customers_orders_items.groupby('customer_unique_id')[['price', 'freight_value']].sum()

In [46]:
# Verificando a quantidade de clientes únicos, se fizermos a diferença com 'values_by_customer', vamos obter os clientes que nunca fizeram pedidos (Que não confirmaram de fato).
olist_customers.customer_unique_id.unique().shape

(96096,)

In [47]:
values_by_customer["total_value"] = values_by_customer["price"] + values_by_customer["freight_value"]
values_by_customer = values_by_customer.reset_index(level=0)
values_by_customer

,customer_unique_id,price,freight_value,total_value
0,0000366f3b9a7992bf8c76cfdf3221e2,129.90,12.00,141.90
1,0000b849f77a49e4a4ce2b2a4ca5be3f,18.90,8.29,27.19
2,0000f46a3911fa3c0805444483337064,69.00,17.22,86.22
3,0000f6ccb0745a6a4b88665a16c9f078,25.99,17.63,43.62
4,0004aac84e0df4da2b147fca70cf8255,180.00,16.89,196.89
...,...,...,...,...
94978,fffcf5a5ff07b0908bd4e2dbc735a684,1570.00,497.42,2067.42
94979,fffea47cd6d3cc0a88bd621562a9d061,64.89,19.69,84.58
94980,ffff371b4d645b6ecea244b27531430a,89.90,22.56,112.46
94981,ffff5962728ec6157033ef9805bacc48,115.00,18.69,133.69


In [48]:
values_by_customer.describe()

,price,freight_value,total_value
count,94983.000000,94983.000000,94983.000000
mean,142.071747,23.595025,165.666772
std,216.074999,22.867488,226.726478
min,0.850000,0.000000,9.590000
25%,47.900000,14.090000,63.100000
50%,89.890000,17.600000,107.900000
75%,155.000000,25.540000,182.930000
max,13440.000000,1794.960000,13664.080000


Feature - Total de pedidos por cliente

In [49]:
customers_orders = customers_orders[(customers_orders.order_status != 'unavailable') & (customers_orders.order_status != 'canceled') & (customers_orders.order_status != 'invoiced')]

In [50]:
quant_orders_by_cust = customers_orders.groupby(by="customer_unique_id").count()["order_id"] 
quant_orders_by_cust = quant_orders_by_cust.reset_index(level=0)


In [51]:
# Total de pedidos por cliente
quant_orders_by_cust

,customer_unique_id,order_id
0,0000366f3b9a7992bf8c76cfdf3221e2,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1
2,0000f46a3911fa3c0805444483337064,1
3,0000f6ccb0745a6a4b88665a16c9f078,1
4,0004aac84e0df4da2b147fca70cf8255,1
...,...,...
94687,fffcf5a5ff07b0908bd4e2dbc735a684,1
94688,fffea47cd6d3cc0a88bd621562a9d061,1
94689,ffff371b4d645b6ecea244b27531430a,1
94690,ffff5962728ec6157033ef9805bacc48,1


In [52]:
# Faturamento total por cliente
values_by_customer

,customer_unique_id,price,freight_value,total_value
0,0000366f3b9a7992bf8c76cfdf3221e2,129.90,12.00,141.90
1,0000b849f77a49e4a4ce2b2a4ca5be3f,18.90,8.29,27.19
2,0000f46a3911fa3c0805444483337064,69.00,17.22,86.22
3,0000f6ccb0745a6a4b88665a16c9f078,25.99,17.63,43.62
4,0004aac84e0df4da2b147fca70cf8255,180.00,16.89,196.89
...,...,...,...,...
94978,fffcf5a5ff07b0908bd4e2dbc735a684,1570.00,497.42,2067.42
94979,fffea47cd6d3cc0a88bd621562a9d061,64.89,19.69,84.58
94980,ffff371b4d645b6ecea244b27531430a,89.90,22.56,112.46
94981,ffff5962728ec6157033ef9805bacc48,115.00,18.69,133.69


Juntar as duas tabelas - 'values_by_customer' e 'quant_orders_by_cust'

In [54]:
customers_info = values_by_customer.merge(quant_orders_by_cust, how="inner", on="customer_unique_id")

In [55]:
customers_info.drop(columns=["price", "freight_value"], inplace=True)
customers_info.rename(columns={"order_id": "quant_orders"}, inplace=True)

In [57]:
customers_info.describe()

,total_value,quant_orders
count,94687.000000,94687.000000
mean,165.498504,1.033806
std,226.542328,0.210731
min,9.590000,1.000000
25%,63.100000,1.000000
50%,107.850000,1.000000
75%,182.860000,1.000000
max,13664.080000,16.000000


Salvando em formato .csv

In [60]:
customers_info.to_csv("../data/processed/customers_info.csv")